# Iterating over collections

A great tutorial for iterators can be found here:

- http://lostella.github.io/blog/2018/07/25/iterative-methods-done-right

This notebook presents some of the examples in the tutorial

### zip over pair of collections

In [35]:
a = 1:5;
b = ["e","d","b","c","a"];

In [36]:
for (p,q) in zip(a,b)
    println("$p $q")
end

1 e
2 d
3 b
4 c
5 a


In [48]:
# A Collection can be bigger than the other
a = 1:10;
b = ["e","d","b","c","a"];

In [39]:
for (p,q) in zip(a,b)
    println("$p $q")
end

1 e
2 d
3 b
4 c
5 a


### Enumerate

In [119]:
a = ["a", "b", "c"];

for (index, value) in enumerate(a)
    println("$index $value")
end

1 a
2 b
3 c


### rest
An iterator that yields the same elements as **`iter`**, but starting at the given **`state`**.



In [123]:
collect(Iterators.rest(["a","b","c","d","e"], 3))

3-element Array{String,1}:
 "c"
 "d"
 "e"

In [124]:
collect(Iterators.rest(["a","b","c","d","e"], 2))

4-element Array{String,1}:
 "b"
 "c"
 "d"
 "e"

### countfrom

In [125]:
for v in Iterators.countfrom(5, 2)
           v > 10 && break
     println(v)
end

5
7
9


### take

In [132]:
a=1:2:10

1:2:9

In [133]:
collect(a)

5-element Array{Int64,1}:
 1
 3
 5
 7
 9

In [134]:
collect(Iterators.take(a,3))

3-element Array{Int64,1}:
 1
 3
 5

### drop

In [136]:
a=1:10

1:10

In [137]:
collect(Iterators.drop(a,4))

6-element Array{Int64,1}:
  5
  6
  7
  8
  9
 10

### Cycle

In [110]:
for (i, v) in enumerate(Iterators.cycle([1,2,3,4]))
print(v)
i > 10 && break
end

12341234123

### repeated

In [113]:
a = Iterators.repeated([1 2], 4);

In [114]:
collect(a)

4-element Array{Array{Int64,2},1}:
 [1 2]
 [1 2]
 [1 2]
 [1 2]

### product

In [143]:
collect(Iterators.product(["a","b"], 1:3))

2×3 Array{Tuple{String,Int64},2}:
 ("a", 1)  ("a", 2)  ("a", 3)
 ("b", 1)  ("b", 2)  ("b", 3)

In [144]:
collect(Iterators.product(["a","b"], 1:5))

2×5 Array{Tuple{String,Int64},2}:
 ("a", 1)  ("a", 2)  ("a", 3)  ("a", 4)  ("a", 5)
 ("b", 1)  ("b", 2)  ("b", 3)  ("b", 4)  ("b", 5)

### Flatten

In [100]:
collect(Iterators.flatten([[1,2,3],[9,10],[21,32,102]]))

8-element Array{Int64,1}:
   1
   2
   3
   9
  10
  21
  32
 102

### partition

In [105]:
x = Iterators.partition(1:10, 3)

Base.Iterators.PartitionIterator{UnitRange{Int64}}(1:10, 3)

In [106]:
collect(x)

4-element Array{Array{Int64,1},1}:
 [1, 2, 3]
 [4, 5, 6]
 [7, 8, 9]
 [10]     

### Filter

In [95]:
x = Iterators.filter(isodd, 1:10)

Base.Iterators.Filter{typeof(isodd),UnitRange{Int64}}(isodd, 1:10)

In [96]:
collect(x)

5-element Array{Int64,1}:
 1
 3
 5
 7
 9

### reverse

In [97]:
x = Iterators.reverse(1:10)

10:-1:1

In [99]:
collect(x)

10-element Array{Int64,1}:
 10
  9
  8
  7
  6
  5
  4
  3
  2
  1

### Partition

In [85]:
collect(Iterators.partition([1,2,3,4,5], 2))

3-element Array{Array{Int64,1},1}:
 [1, 2]
 [3, 4]
 [5]   

In [138]:
collect(Iterators.partition([1,2,3,4,5], 3))

2-element Array{Array{Int64,1},1}:
 [1, 2, 3]
 [4, 5]   

# Defining custom iteration procedures

In [ ]:
struct MyType{T}
    initial_state::T
    state::T
end

In [ ]:
seq = FibonacciIterable(0,0)

In [ ]:
import Base: iterate
iterate(x::MyType) = x.initial_state, x.state
iterate(x::MyType, state) = state[1], (sum(state))

### Fibonacci example


For instance, suppose we want to compute the following generalized Fibonacci sequence:

```
start with s_0=0, s_1=1

F_0 = s_0,
F_1 = s_1,
F_n = F_{n-1} +  F_{n-2}
```

In [1]:
struct FibonacciIterable{I}
    s0::I
    s1::I
end

In [2]:
fib = FibonacciIterable(0,1)

FibonacciIterable{Int64}(0, 1)

In order to unroll the sequence and compute each element, one only needs to keep track of the pair $(F_{n-1}, F_{n})$ of the two most recent elements: that will be the state of our iteration. Based on that, we need to define two methods for the iterate function:

- `iterate(iter::FibonacciIterable)` returning the pair (F0, state0) containing the first element in the sequence and initial state respectively;
- `iterate(iter::FibonacciIterable, state)` returning the pair (F, newstate) of the next element F in the sequence (given state) and the updated state newstateiterate(iter::FibonacciIterable, state) returning the pair (F, newstate) of the next element F in the sequence (given state) and the updated state newstate-.

As soon as the sequence is over, iterate should return nothing (Julia's "none" value). This is never the case for our Fibonacci sequences, since they're infinite. Note that FibonacciIterable objects are immutable: we would like them not to change as we iterate, since they identify the sequence which is being produced. Instead, all mutations should occur in state updates.

The following definitions give exactly the above described generalized Fibonacci sequence:



In [3]:
import Base: iterate
iterate(iter::FibonacciIterable) = iter.s0, (iter.s0, iter.s1)
iterate(iter::FibonacciIterable, state) = state[2], (state[2], sum(state))

iterate (generic function with 204 methods)

In [14]:
fib0,s  = iterate(fib)

(0, (0, 1))

In [17]:
fib2 = iterate(s)

(0, 2)

In [16]:
fib3 = iterate(s)

(0, 2)

In [ ]:
iterate()

In [ ]:
for F in FibonacciIterable(BigInt(0), BigInt(1))
    println(F)
    if F > 50 break end
end

In [ ]:
 BigInt(1)

## Conjugate gradient method

The conjugate gradient (CG) method solves linear systems

$$ Ax = b $$

where $A\in\mathbb{R}^{n\times n}$ is a positive semidefinite, symmetric matrix. It is particularly useful when $n$ is very large and $$A$$ is sparse, in which case direct methods (Cholesky factorization) are computationally prohibitive. Instead, CG works by only applying matrix-vector products with $A$. Given an initial guess $x_0$, the method produces a sequence $x_k$ of solution approximations according to the following recurrence:

Initialize $ r_0 = p_0 = Ax_0 - b $.

- For any $k > 0$ do

$$ \begin{align} 
\alpha_k &= \frac{|r_k|^2}{\langle p_k, Ap_k \rangle}, \\ 
x_{k+1} &= x_{k} + \alpha_{k} p_{k}, \\
r_{k+1} &= r_{k} + \alpha_{k} A p_{k}, \\ 
p_{k+1} &= r_{k+1} + \frac{|r_{k+1}|^2}{|r_{k}|^2}p_{k}. 
\end{align}$$





Let's now translate CG into a Julia iterable type. 
    
The iteration is completely determined by matrix $A$, vector $b$,
and the initial guess $x_0$, so those will compose our iterable objects:

In [ ]:
struct CGIterable{TA, Tb, Tx}
    A::TA
    b::Tb
    x0::Tx
end

cgiterable(A::TA, b::Tb; x0::Tx=nothing) where {TA, Tb, Tx} = CGIterable{TA, Tb, Tx}(A, b, x0)


In the cgiterable constructor we allow for x0 to be nothing: 
in this case we will assume the initial point to be the zero vector, 
and save ourselves one matrix-vector product. The state of the iteration is composed of vectors $x_k$, $r_k$, and $p_k$. 

In addition to that, we will make room for additional stuff, so as to reuse all possible memory and avoid allocations: vector $A p_k$ and scalars $|r_k|^2$ and $|r_{k+1}|^2$. Overall, the state for CG is a bit more complex than in the Fibonacci example, and keeping it in a Tuple would be impractical. So let's give things a name by defining a custom type for the state:

```julia
mutable struct CGState{R, Tb}
    x::Tb
    r::Tb
    rs::R
    rsprev::R
    p::Tb
    Ap::Tb
end
```

Note that CGState is defined as mutable: this is because we will overwrite the iteration state rather than allocate new objects, again for efficiency reasons.

The actual computation is carried out by the iterate function:

```julia
using LinearAlgebra
import Base: iterate

function iterate(iter::CGIterable{TA, Tb, Tx}) where
    {R, TA, Tb <: AbstractVector{R}, Tx}
    if iter.x0 === nothing
        x = zero(iter.b)
        r = copy(iter.b)
    else
        x = copy(iter.x0)
        r = iter.A*x
        r .*= -one(R)
        r .+= iter.b
    end
    rs = dot(r, r)
    p = copy(r)
    Ap = similar(r)
    state = CGState{R, Tb}(x, r, rs, zero(rs), p, Ap)
    return state, state
end

function iterate(iter::CGIterable{TA, Tb, Tx}, state::CGState{R, Tb}) where
    {R, TA, Tb <: AbstractVector{R}, Tx}
    mul!(state.Ap, iter.A, state.p)
    alpha = state.rs / dot(state.p, state.Ap)
    state.x .+= alpha .* state.p
    state.r .-= alpha .* state.Ap
    state.rsprev = state.rs
    state.rs = dot(state.r, state.r)
    state.p .= state.r .+ (state.rs / state.rsprev) .* state.p
    return state, state
end
```

Rather than the sequence of $x_k$, we yield the sequence of states of the algorithm, since that contains all information that may be needed when experimenting with the algorithm (including $x_k$ itself).

